In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [63]:
price_and_demand = pd.read_csv("price_and_demand.csv")
price_and_demand

,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE
0,VIC1,1/11/2022 0:30,4178.18,8.94,TRADE
1,VIC1,1/11/2022 1:00,4086.02,0.14,TRADE
2,VIC1,1/11/2022 1:30,4033.37,0.02,TRADE
3,VIC1,1/11/2022 2:00,3985.64,0.00,TRADE
4,VIC1,1/11/2022 2:30,3885.64,0.02,TRADE
...,...,...,...,...,...
8347,VIC1,23/04/2023 22:00,4717.55,84.09,TRADE
8348,VIC1,23/04/2023 22:30,4520.39,81.25,TRADE
8349,VIC1,23/04/2023 23:00,4472.49,93.97,TRADE
8350,VIC1,23/04/2023 23:30,4621.36,83.96,TRADE


In [64]:
# Check datatype of the dataframe
price_and_demand.dtypes

REGION             object
SETTLEMENTDATE     object
TOTALDEMAND       float64
RRP               float64
PERIODTYPE         object
dtype: object

In [65]:
# check if there is any mising data
price_and_demand.isna().sum()

REGION            0
SETTLEMENTDATE    0
TOTALDEMAND       0
RRP               0
PERIODTYPE        0
dtype: int64

In [66]:
# Convert "SETTLEMENTDATE" column to datetime
price_and_demand["SETTLEMENTDATE"] = pd.to_datetime(price_and_demand["SETTLEMENTDATE"], format="%d/%m/%Y %H:%M")

In [67]:
# Extract date from "SETTLEMENTDATE" and create a new 'Date' column
price_and_demand['Date'] = price_and_demand['SETTLEMENTDATE'].dt.date

In [68]:
# Sampling 20 examples
price_and_demand.sample(20)

,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE,Date
5180,VIC1,2023-02-16 22:30:00,5465.80,110.20,TRADE,2023-02-16
7308,VIC1,2023-04-02 06:30:00,3997.25,141.00,TRADE,2023-04-02
888,VIC1,2022-11-19 12:30:00,4130.71,-34.14,TRADE,2022-11-19
6807,VIC1,2023-03-22 20:00:00,5352.95,90.26,TRADE,2023-03-22
7671,VIC1,2023-04-09 20:00:00,5082.40,90.29,TRADE,2023-04-09
7978,VIC1,2023-04-16 05:30:00,3499.85,57.92,TRADE,2023-04-16
2406,VIC1,2022-12-21 03:30:00,3901.61,79.40,TRADE,2022-12-21
2681,VIC1,2022-12-26 21:00:00,5336.57,84.33,TRADE,2022-12-26
2192,VIC1,2022-12-16 16:30:00,3902.56,-38.00,TRADE,2022-12-16
8135,VIC1,2023-04-19 12:00:00,4099.56,-2.21,TRADE,2023-04-19


In [69]:
# Calculate maximum daily energy demand and average rrp
max_demand = price_and_demand.groupby("Date")['TOTALDEMAND'].max()
print(type(max_demand))

<class 'pandas.core.series.Series'>


In [70]:
mean_rrp = price_and_demand.groupby("Date")['RRP'].mean().round(2)
mean_rrp

Date
2022-11-01     -3.76
2022-11-02     -2.52
2022-11-03     63.21
2022-11-04    110.94
2022-11-05     72.96
               ...  
2023-04-20    130.71
2023-04-21    131.46
2023-04-22     80.39
2023-04-23     63.21
2023-04-24     63.77
Name: RRP, Length: 175, dtype: float64

In [71]:
# Importing Weather Dataset
weather = pd.read_csv("weather.csv", encoding='latin-1',dayfirst = True)
weather

,Location,Date,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,...,9am cloud amount (oktas),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
0,"Melbourne (Olympic Park), Victoria",1/11/2022,8.5,13.3,3.0,NaN,NaN,SW,44.0,11:36,...,NaN,NW,13,991.3,13.2,59.0,NaN,WNW,11.0,991.5
1,"Melbourne (Olympic Park), Victoria",2/11/2022,6.9,15.7,2.8,NaN,NaN,SSW,43.0,11:33,...,NaN,W,9,1006.7,15.1,54.0,NaN,SSW,13.0,1008.4
2,"Melbourne (Olympic Park), Victoria",3/11/2022,9.1,15.4,0.4,NaN,NaN,SSW,31.0,8:07,...,NaN,SSW,9,1019.2,13.8,67.0,NaN,SSW,9.0,1021.3
3,"Melbourne (Olympic Park), Victoria",4/11/2022,10.2,17.8,0.2,NaN,NaN,S,24.0,14:50,...,NaN,WSW,6,1028.1,16.9,56.0,NaN,SW,9.0,1026.6
4,"Melbourne (Olympic Park), Victoria",5/11/2022,11.8,22.7,0.0,NaN,NaN,N,31.0,11:58,...,NaN,N,9,1026.3,18.8,65.0,NaN,SW,11.0,1023.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,"Melbourne (Olympic Park), Victoria",20/04/2023,7.8,18.5,0.0,NaN,NaN,SSW,17.0,12:47,...,NaN,NNE,7,1026.8,17.5,58.0,NaN,SW,4.0,1025.5
171,"Melbourne (Olympic Park), Victoria",21/04/2023,11.3,19.0,0.0,NaN,NaN,SSW,13.0,10:48,...,NaN,NE,6,1030.5,18.6,64.0,NaN,SSE,2.0,1029.2
172,"Melbourne (Olympic Park), Victoria",22/04/2023,14.6,19.2,0.0,NaN,NaN,SSW,22.0,16:09,...,NaN,SW,6,1031.9,18.8,64.0,NaN,SSW,11.0,1029.7
173,"Melbourne (Olympic Park), Victoria",23/04/2023,14.8,19.0,0.0,NaN,NaN,SSW,26.0,15:45,...,NaN,SSE,7,1034.4,17.8,71.0,NaN,SSW,13.0,1032.4


In [72]:
weather.dtypes

Location                              object
Date                                  object
Minimum temperature (°C)             float64
Maximum temperature (°C)             float64
Rainfall (mm)                        float64
Evaporation (mm)                     float64
Sunshine (hours)                     float64
Direction of maximum wind gust        object
Speed of maximum wind gust (km/h)    float64
Time of maximum wind gust             object
9am Temperature (°C)                 float64
9am relative humidity (%)              int64
9am cloud amount (oktas)             float64
9am wind direction                    object
9am wind speed (km/h)                 object
9am MSL pressure (hPa)               float64
3pm Temperature (°C)                 float64
3pm relative humidity (%)            float64
3pm cloud amount (oktas)             float64
3pm wind direction                    object
3pm wind speed (km/h)                float64
3pm MSL pressure (hPa)               float64
dtype: obj

In [73]:
#Removing entirely empty columns
weather = weather.dropna(axis = 1, how = 'all')
weather

,Location,Date,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,9am Temperature (°C),9am relative humidity (%),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
0,"Melbourne (Olympic Park), Victoria",1/11/2022,8.5,13.3,3.0,SW,44.0,11:36,12.0,64,NW,13,991.3,13.2,59.0,WNW,11.0,991.5
1,"Melbourne (Olympic Park), Victoria",2/11/2022,6.9,15.7,2.8,SSW,43.0,11:33,11.0,65,W,9,1006.7,15.1,54.0,SSW,13.0,1008.4
2,"Melbourne (Olympic Park), Victoria",3/11/2022,9.1,15.4,0.4,SSW,31.0,8:07,11.5,70,SSW,9,1019.2,13.8,67.0,SSW,9.0,1021.3
3,"Melbourne (Olympic Park), Victoria",4/11/2022,10.2,17.8,0.2,S,24.0,14:50,12.3,84,WSW,6,1028.1,16.9,56.0,SW,9.0,1026.6
4,"Melbourne (Olympic Park), Victoria",5/11/2022,11.8,22.7,0.0,N,31.0,11:58,14.0,78,N,9,1026.3,18.8,65.0,SW,11.0,1023.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,"Melbourne (Olympic Park), Victoria",20/04/2023,7.8,18.5,0.0,SSW,17.0,12:47,12.4,72,NNE,7,1026.8,17.5,58.0,SW,4.0,1025.5
171,"Melbourne (Olympic Park), Victoria",21/04/2023,11.3,19.0,0.0,SSW,13.0,10:48,14.8,78,NE,6,1030.5,18.6,64.0,SSE,2.0,1029.2
172,"Melbourne (Olympic Park), Victoria",22/04/2023,14.6,19.2,0.0,SSW,22.0,16:09,15.2,84,SW,6,1031.9,18.8,64.0,SSW,11.0,1029.7
173,"Melbourne (Olympic Park), Victoria",23/04/2023,14.8,19.0,0.0,SSW,26.0,15:45,16.7,79,SSE,7,1034.4,17.8,71.0,SSW,13.0,1032.4


In [74]:
#Setting Date format to match
weather = weather.copy()
weather["Date"] = pd.to_datetime(weather['Date'], format="%d/%m/%Y")
weather.head()

,Location,Date,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,9am Temperature (°C),9am relative humidity (%),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
0,"Melbourne (Olympic Park), Victoria",2022-11-01,8.5,13.3,3.0,SW,44.0,11:36,12.0,64,NW,13,991.3,13.2,59.0,WNW,11.0,991.5
1,"Melbourne (Olympic Park), Victoria",2022-11-02,6.9,15.7,2.8,SSW,43.0,11:33,11.0,65,W,9,1006.7,15.1,54.0,SSW,13.0,1008.4
2,"Melbourne (Olympic Park), Victoria",2022-11-03,9.1,15.4,0.4,SSW,31.0,8:07,11.5,70,SSW,9,1019.2,13.8,67.0,SSW,9.0,1021.3
3,"Melbourne (Olympic Park), Victoria",2022-11-04,10.2,17.8,0.2,S,24.0,14:50,12.3,84,WSW,6,1028.1,16.9,56.0,SW,9.0,1026.6
4,"Melbourne (Olympic Park), Victoria",2022-11-05,11.8,22.7,0.0,N,31.0,11:58,14.0,78,N,9,1026.3,18.8,65.0,SW,11.0,1023.2


In [78]:
#Setting Date as the ID Column
weather_dateindex = weather.set_index('Date')
weather_dateindex

,Location,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,9am Temperature (°C),9am relative humidity (%),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
Date,,,,,,,,,,,,,,,,,
2022-11-01,"Melbourne (Olympic Park), Victoria",8.5,13.3,3.0,SW,44.0,11:36,12.0,64,NW,13,991.3,13.2,59.0,WNW,11.0,991.5
2022-11-02,"Melbourne (Olympic Park), Victoria",6.9,15.7,2.8,SSW,43.0,11:33,11.0,65,W,9,1006.7,15.1,54.0,SSW,13.0,1008.4
2022-11-03,"Melbourne (Olympic Park), Victoria",9.1,15.4,0.4,SSW,31.0,8:07,11.5,70,SSW,9,1019.2,13.8,67.0,SSW,9.0,1021.3
2022-11-04,"Melbourne (Olympic Park), Victoria",10.2,17.8,0.2,S,24.0,14:50,12.3,84,WSW,6,1028.1,16.9,56.0,SW,9.0,1026.6
2022-11-05,"Melbourne (Olympic Park), Victoria",11.8,22.7,0.0,N,31.0,11:58,14.0,78,N,9,1026.3,18.8,65.0,SW,11.0,1023.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-20,"Melbourne (Olympic Park), Victoria",7.8,18.5,0.0,SSW,17.0,12:47,12.4,72,NNE,7,1026.8,17.5,58.0,SW,4.0,1025.5
2023-04-21,"Melbourne (Olympic Park), Victoria",11.3,19.0,0.0,SSW,13.0,10:48,14.8,78,NE,6,1030.5,18.6,64.0,SSE,2.0,1029.2
2023-04-22,"Melbourne (Olympic Park), Victoria",14.6,19.2,0.0,SSW,22.0,16:09,15.2,84,SW,6,1031.9,18.8,64.0,SSW,11.0,1029.7


In [79]:
"""Data Cleaning across entire dataframe"""
#Replacing all occurrences of 'Calm' windspeed with 0 km/h
weather_nocalm = weather_dateindex.replace("Calm",0)
weather_nocalm

,Location,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,9am Temperature (°C),9am relative humidity (%),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
Date,,,,,,,,,,,,,,,,,
2022-11-01,"Melbourne (Olympic Park), Victoria",8.5,13.3,3.0,SW,44.0,11:36,12.0,64,NW,13,991.3,13.2,59.0,WNW,11.0,991.5
2022-11-02,"Melbourne (Olympic Park), Victoria",6.9,15.7,2.8,SSW,43.0,11:33,11.0,65,W,9,1006.7,15.1,54.0,SSW,13.0,1008.4
2022-11-03,"Melbourne (Olympic Park), Victoria",9.1,15.4,0.4,SSW,31.0,8:07,11.5,70,SSW,9,1019.2,13.8,67.0,SSW,9.0,1021.3
2022-11-04,"Melbourne (Olympic Park), Victoria",10.2,17.8,0.2,S,24.0,14:50,12.3,84,WSW,6,1028.1,16.9,56.0,SW,9.0,1026.6
2022-11-05,"Melbourne (Olympic Park), Victoria",11.8,22.7,0.0,N,31.0,11:58,14.0,78,N,9,1026.3,18.8,65.0,SW,11.0,1023.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-20,"Melbourne (Olympic Park), Victoria",7.8,18.5,0.0,SSW,17.0,12:47,12.4,72,NNE,7,1026.8,17.5,58.0,SW,4.0,1025.5
2023-04-21,"Melbourne (Olympic Park), Victoria",11.3,19.0,0.0,SSW,13.0,10:48,14.8,78,NE,6,1030.5,18.6,64.0,SSE,2.0,1029.2
2023-04-22,"Melbourne (Olympic Park), Victoria",14.6,19.2,0.0,SSW,22.0,16:09,15.2,84,SW,6,1031.9,18.8,64.0,SSW,11.0,1029.7


In [81]:
#Replacing all wind directions with True Bearing quantities
weather_bearings = weather_nocalm.replace(['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW'],
                                          [0,22.5,45,67.5,90,112.5,135,157.5,180,202.5,225,247.5,270,292.5,315,337.5])
#Renaming Columns with True Bearing Units
weather_bearings.rename(columns = {'Direction of maximum wind gust ':'Direction of maximum wind gust (°T)', '9am wind direction':'9am wind direction (°T)','3pm wind direction':'3pm wind direction (°T)'},inplace = True)

#Data Cleaning Complete
processed_data = weather_bearings

processed_data

,Location,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Direction of maximum wind gust (°T),Speed of maximum wind gust (km/h),Time of maximum wind gust,9am Temperature (°C),9am relative humidity (%),9am wind direction (°T),9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm wind direction (°T),3pm wind speed (km/h),3pm MSL pressure (hPa)
Date,,,,,,,,,,,,,,,,,
2022-11-01,"Melbourne (Olympic Park), Victoria",8.5,13.3,3.0,225.0,44.0,11:36,12.0,64,315,13,991.3,13.2,59.0,292.5,11.0,991.5
2022-11-02,"Melbourne (Olympic Park), Victoria",6.9,15.7,2.8,202.5,43.0,11:33,11.0,65,270,9,1006.7,15.1,54.0,202.5,13.0,1008.4
2022-11-03,"Melbourne (Olympic Park), Victoria",9.1,15.4,0.4,202.5,31.0,8:07,11.5,70,202.5,9,1019.2,13.8,67.0,202.5,9.0,1021.3
2022-11-04,"Melbourne (Olympic Park), Victoria",10.2,17.8,0.2,180.0,24.0,14:50,12.3,84,247.5,6,1028.1,16.9,56.0,225.0,9.0,1026.6
2022-11-05,"Melbourne (Olympic Park), Victoria",11.8,22.7,0.0,0.0,31.0,11:58,14.0,78,0,9,1026.3,18.8,65.0,225.0,11.0,1023.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-20,"Melbourne (Olympic Park), Victoria",7.8,18.5,0.0,202.5,17.0,12:47,12.4,72,22.5,7,1026.8,17.5,58.0,225.0,4.0,1025.5
2023-04-21,"Melbourne (Olympic Park), Victoria",11.3,19.0,0.0,202.5,13.0,10:48,14.8,78,45,6,1030.5,18.6,64.0,157.5,2.0,1029.2
2023-04-22,"Melbourne (Olympic Park), Victoria",14.6,19.2,0.0,202.5,22.0,16:09,15.2,84,225,6,1031.9,18.8,64.0,202.5,11.0,1029.7


In [82]:
#Selecting key features based on 'expert' insight and desired focus variables
selected_features = ["Minimum temperature (°C)", "Maximum temperature (°C)", "Rainfall (mm)","Speed of maximum wind gust (km/h)","9am relative humidity (%)","3pm relative humidity (%)", "9am wind speed (km/h)","3pm wind speed (km/h)", "9am MSL pressure (hPa)", "3pm MSL pressure (hPa)"]
# selected_features = ["Maximum temperature (°C)", "3pm relative humidity (%)"]
selected_features

selected_data = processed_data[selected_features]

#Checking for missing values across selected features
missing_value = processed_data[selected_features].isnull()
missing_value

#Removing data points with missing selected features
selected_data = processed_data[selected_features].dropna()
predictor_data = processed_data[selected_features].dropna()
predictor_data

,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Speed of maximum wind gust (km/h),9am relative humidity (%),3pm relative humidity (%),9am wind speed (km/h),3pm wind speed (km/h),9am MSL pressure (hPa),3pm MSL pressure (hPa)
Date,,,,,,,,,,
2022-11-01,8.5,13.3,3.0,44.0,64,59.0,13,11.0,991.3,991.5
2022-11-02,6.9,15.7,2.8,43.0,65,54.0,9,13.0,1006.7,1008.4
2022-11-03,9.1,15.4,0.4,31.0,70,67.0,9,9.0,1019.2,1021.3
2022-11-04,10.2,17.8,0.2,24.0,84,56.0,6,9.0,1028.1,1026.6
2022-11-05,11.8,22.7,0.0,31.0,78,65.0,9,11.0,1026.3,1023.2
...,...,...,...,...,...,...,...,...,...,...
2023-04-19,14.4,17.5,1.2,28.0,66,53.0,9,9.0,1022.0,1022.1
2023-04-20,7.8,18.5,0.0,17.0,72,58.0,7,4.0,1026.8,1025.5
2023-04-21,11.3,19.0,0.0,13.0,78,64.0,6,2.0,1030.5,1029.2


In [83]:
# Converting all data to float64
selected_data = selected_data.astype(float)
selected_data

predictor_data = predictor_data.astype(float)
predictor_data

,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Speed of maximum wind gust (km/h),9am relative humidity (%),3pm relative humidity (%),9am wind speed (km/h),3pm wind speed (km/h),9am MSL pressure (hPa),3pm MSL pressure (hPa)
Date,,,,,,,,,,
2022-11-01,8.5,13.3,3.0,44.0,64.0,59.0,13.0,11.0,991.3,991.5
2022-11-02,6.9,15.7,2.8,43.0,65.0,54.0,9.0,13.0,1006.7,1008.4
2022-11-03,9.1,15.4,0.4,31.0,70.0,67.0,9.0,9.0,1019.2,1021.3
2022-11-04,10.2,17.8,0.2,24.0,84.0,56.0,6.0,9.0,1028.1,1026.6
2022-11-05,11.8,22.7,0.0,31.0,78.0,65.0,9.0,11.0,1026.3,1023.2
...,...,...,...,...,...,...,...,...,...,...
2023-04-19,14.4,17.5,1.2,28.0,66.0,53.0,9.0,9.0,1022.0,1022.1
2023-04-20,7.8,18.5,0.0,17.0,72.0,58.0,7.0,4.0,1026.8,1025.5
2023-04-21,11.3,19.0,0.0,13.0,78.0,64.0,6.0,2.0,1030.5,1029.2


In [84]:
# Introducing Target Variables
selected_data["Average RRP"] = mean_rrp
selected_data["Maximum Energy Demand"] = max_demand
selected_data

,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Speed of maximum wind gust (km/h),9am relative humidity (%),3pm relative humidity (%),9am wind speed (km/h),3pm wind speed (km/h),9am MSL pressure (hPa),3pm MSL pressure (hPa),Average RRP,Maximum Energy Demand
Date,,,,,,,,,,,,
2022-11-01,8.5,13.3,3.0,44.0,64.0,59.0,13.0,11.0,991.3,991.5,-3.76,5627.52
2022-11-02,6.9,15.7,2.8,43.0,65.0,54.0,9.0,13.0,1006.7,1008.4,-2.52,6018.43
2022-11-03,9.1,15.4,0.4,31.0,70.0,67.0,9.0,9.0,1019.2,1021.3,63.21,6004.84
2022-11-04,10.2,17.8,0.2,24.0,84.0,56.0,6.0,9.0,1028.1,1026.6,110.94,5757.62
2022-11-05,11.8,22.7,0.0,31.0,78.0,65.0,9.0,11.0,1026.3,1023.2,72.96,4783.60
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-19,14.4,17.5,1.2,28.0,66.0,53.0,9.0,9.0,1022.0,1022.1,102.33,6042.88
2023-04-20,7.8,18.5,0.0,17.0,72.0,58.0,7.0,4.0,1026.8,1025.5,130.71,6084.21
2023-04-21,11.3,19.0,0.0,13.0,78.0,64.0,6.0,2.0,1030.5,1029.2,131.46,5903.60
